In [1]:
# Imports (numpy, pandas, matplotlib, seaborn imported automatically)
from etherscan import Etherscan
from BondingCurveNexus import config
from scipy.stats import lognorm

In [2]:
# activate etherscan client
eth_client = Etherscan(config.etherscan_api)

eth_price = float(eth_client.get_eth_last_price()['ethusd'])

In [3]:
eth_price

1077.66

# Investigate patterns in cover amounts and claim frequencies

## Bring in files and convert datetimes

In [11]:
# bring in csvs

# historic active cover amount
active_cover_amount = pd.read_csv('../raw_data/active_cover_amount.csv')
active_cover_amount.columns = ['date', 'USD', 'ETH']
active_cover_amount['date'] = pd.to_datetime(active_cover_amount['date']).dt.date

# claims history
claims_history = pd.read_csv('../raw_data/all_claims.csv')
claims_history['timestamp'] = pd.to_datetime(claims_history['timestamp'])

# cover history
cover_history = pd.read_csv('../raw_data/all_covers.csv')
cover_history['start_time'] = pd.to_datetime(cover_history['start_time'])
cover_history['end_time'] = pd.to_datetime(cover_history['end_time'])

## Investigate Distribution of Daily changes in Active cover amount

In [5]:
active_cover_amount = active_cover_amount.groupby('date').max()

In [6]:
active_cover_amount['ETH_change_perc'] = active_cover_amount['ETH'] / active_cover_amount['ETH'].shift(1) - 1
active_cover_amount['ETH_change_val'] = active_cover_amount['ETH'] - active_cover_amount['ETH'].shift(1)

In [7]:
active_cover_amount

,USD,ETH,ETH_change_perc,ETH_change_val
date,,,,
2019-07-12,9.775000e+02,3.549383,NaN,NaN
2019-07-13,2.807950e+04,103.606745,28.190074,100.057362
2019-07-14,7.119130e+04,304.197325,1.936076,200.590580
2019-07-15,1.085736e+05,472.018185,0.551684,167.820860
2019-07-16,2.327708e+05,1161.878556,1.461512,689.860371
...,...,...,...,...
2022-06-06,3.219952e+08,172156.440888,-0.006095,-1055.660376
2022-06-07,2.976967e+08,167008.480992,-0.029903,-5147.959896
2022-06-08,3.012918e+08,168740.869302,0.010373,1732.388310


In [8]:
active_cover_amount[-240:]

,USD,ETH,ETH_change_perc,ETH_change_val
date,,,,
2021-10-14,4.316843e+08,114637.423597,-0.065447,-8028.045501
2021-10-15,4.637110e+08,119904.621381,0.045947,5267.197784
2021-10-16,4.721258e+08,123408.207362,0.029220,3503.585981
2021-10-17,4.784494e+08,123726.900447,0.002582,318.693085
2021-10-18,4.765855e+08,124704.274779,0.007899,977.374332
...,...,...,...,...
2022-06-06,3.219952e+08,172156.440888,-0.006095,-1055.660376
2022-06-07,2.976967e+08,167008.480992,-0.029903,-5147.959896
2022-06-08,3.012918e+08,168740.869302,0.010373,1732.388310


In [29]:
active_cover_amount[-240:]['ETH_change_val'].describe()

count      240.000000
mean       230.231857
std       4747.128874
min     -19091.121391
25%      -1501.448847
50%         15.189586
75%       2625.313310
max      16371.068444
Name: ETH_change_val, dtype: float64

## Investigate premium income & cover drop-off

In [ ]:
cover_history['start_time'] = pd.to_datetime(cover_history['start_time']).dt.date
cover_history['end_time'] = pd.to_datetime(cover_history['end_time']).dt.date

In [5]:
cover_history

,block_number,start_time,premium,address,cover_id,end_time,currency,amount,project,amount_usd,premium_usd
0,8667271,2019-10-03,0.012991,0xc0a47dfe034b400b47bdad5fecda2621de6c4d95,84,2020-10-02,ETH,1.0,Uniswap v1,1073.59,13.947117
1,8759426,2019-10-17,0.064066,0xc0a47dfe034b400b47bdad5fecda2621de6c4d95,93,2020-03-15,ETH,12.0,Uniswap v1,12883.08,68.780304
2,8873386,2019-11-04,0.064066,0xc0a47dfe034b400b47bdad5fecda2621de6c4d95,98,2020-04-02,ETH,12.0,Uniswap v1,12883.08,68.780304
3,8895454,2019-11-08,0.074743,0xc0a47dfe034b400b47bdad5fecda2621de6c4d95,100,2020-04-06,ETH,14.0,Uniswap v1,15030.26,80.243688
4,8935109,2019-11-14,0.172977,0xc0a47dfe034b400b47bdad5fecda2621de6c4d95,106,2020-02-12,ETH,54.0,Uniswap v1,57973.86,185.706821
...,...,...,...,...,...,...,...,...,...,...,...
7516,11785120,2021-02-03,1.127023,0xa4c8d221d8bb851f83aadd0223a8900a6921a349,3150,2022-02-03,ETH,7.0,Set Protocol v2,7515.13,1209.960722
7517,11812010,2021-02-07,4.015933,0xa4c8d221d8bb851f83aadd0223a8900a6921a349,3198,2022-02-07,ETH,25.0,Set Protocol v2,26839.75,4311.465267
7518,11965147,2021-03-03,0.134255,0xa4c8d221d8bb851f83aadd0223a8900a6921a349,3531,2021-06-01,ETH,10.0,Set Protocol v2,10735.90,144.134638
7519,12199390,2021-04-08,0.102505,0xa4c8d221d8bb851f83aadd0223a8900a6921a349,3818,2021-10-05,ETH,8.0,Set Protocol v2,8588.72,110.048486


In [10]:
cover_history['premium_eth'] = np.where(cover_history['currency'] == 'DAI',
                                         cover_history['premium']/eth_price, cover_history['premium'])
cover_history['amount_eth'] = np.where(cover_history['currency'] == 'DAI',
                                         cover_history['amount']/eth_price, cover_history['amount'])

In [11]:
cover_history['rate'] = cover_history['premium_eth']/cover_history['amount_eth']

In [12]:
cover_history['rate'].describe()

count    7467.000000
mean        0.009966
std         0.014127
min         0.001068
25%         0.002136
50%         0.006407
75%         0.012813
max         0.447375
Name: rate, dtype: float64

### Premium & Amount History (since June 2020)

In [13]:
# group by day
premium_history = cover_history.groupby('start_time')[['premium_eth']].sum().reset_index()
amount_history = cover_history.groupby('start_time')[['amount_eth']].sum().reset_index()

In [14]:
premium_history = premium_history.iloc[153:]
amount_history = amount_history.iloc[153:]

In [15]:
premium_history = premium_history.set_index('start_time')
amount_history = amount_history.set_index('start_time')

In [16]:
# fill in empty dates with no cover buys
idx = pd.date_range('2020-06-01', '2022-06-10')
premium_history = premium_history.reindex(idx, fill_value=0)
amount_history = amount_history.reindex(idx, fill_value=0)

In [17]:
premium_history.describe()

,premium_eth
count,740.000000
mean,23.327086
std,142.862217
min,0.000000
25%,2.319041
50%,7.551642
75%,18.817329
max,3757.889588


In [19]:
amount_history[500:]

,amount_eth
2021-10-14,1384.428193
2021-10-15,7732.721091
2021-10-16,11555.317320
2021-10-17,295.000000
2021-10-18,5011.961723
...,...
2022-06-06,3767.210540
2022-06-07,2357.252038
2022-06-08,5305.316515
2022-06-09,884.191855


In [18]:
amount_history[500:].describe()

,amount_eth
count,240.000000
mean,4108.938137
std,5429.763931
min,0.000000
25%,717.171710
50%,1956.533748
75%,5655.821723
max,30212.182160


### Drop-off in cover amounts (since 14 Oct 2021)


In [20]:
# group by day
drop_history = cover_history.groupby('end_time')[['amount_eth']].sum().reset_index()
drop_history = drop_history.iloc[569:807]
drop_history = drop_history.set_index('end_time')
idx = pd.date_range('2021-10-14', '2022-06-10')
drop_history = drop_history.reindex(idx, fill_value=0)

In [21]:
drop_history

,amount_eth
2021-10-14,1462.750746
2021-10-15,232.040089
2021-10-16,1291.981851
2021-10-17,338.510886
2021-10-18,1133.082425
...,...
2022-06-06,17411.205743
2022-06-07,197.000000
2022-06-08,1148.981851
2022-06-09,601.868429


In [22]:
drop_history.mean()

amount_eth    3802.920618
dtype: float64

## Investigate frequency and size of accepted claims

In [23]:
# filter accepted claims only
accepted_claims = claims_history[claims_history['verdict'] == 'Accepted']

# work out days since first claim
days_since_first_claim = (pd.Timestamp.now() - accepted_claims['timestamp'].iloc[0]).days

# count number of claims and divide by number of days
avg_claim_freq = accepted_claims['verdict'].count() / days_since_first_claim

avg_claim_freq 

0.03076923076923077

In [24]:
accepted_claims['amount_eth'] = np.where(accepted_claims['currency'] == 'DAI',
                                         accepted_claims['amount']/eth_price, accepted_claims['amount'])

/tmp/ipykernel_1303/834324826.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  accepted_claims['amount_eth'] = np.where(accepted_claims['currency'] == 'DAI',


In [25]:
accepted_claims

,timestamp,cover_id,block_number,verdict,claim_id,project,amount_usd,amount,currency,start_time,amount_eth
0,2020-02-18 17:01:18,152,9508333,Accepted,2,bZx v1,8050.76,4.0,ETH,2020-02-03 19:58:33,4.000000
2,2020-02-19 23:55:22,158,9516760,Accepted,5,bZx v1,2598.70,2600.0,DAI,2020-02-05 01:27:47,2.180549
3,2020-02-18 17:07:50,121,9508364,Accepted,3,bZx v1,29985.00,30000.0,DAI,2019-12-10 15:49:23,25.160187
69,2021-02-04 23:20:43,729,11792745,Accepted,72,Yearn Finance (all vaults),12076.14,6.0,ETH,2020-08-24 17:36:23,6.000000
70,2021-02-05 02:47:32,278,11793668,Accepted,73,Yearn Finance (all vaults),1399.30,1400.0,DAI,2020-04-16 00:45:45,1.174142
72,2021-02-05 02:49:38,616,11793678,Accepted,74,Yearn Finance (all vaults),48304.56,24.0,ETH,2020-08-16 10:02:18,24.000000
73,2021-02-05 03:19:09,203,11793813,Accepted,75,Yearn Finance (all vaults),8995.50,9000.0,DAI,2020-02-20 04:33:53,7.548056
74,2021-02-05 17:42:46,652,11797703,Accepted,77,Yearn Finance (all vaults),12076.14,6.0,ETH,2020-08-17 21:28:10,6.000000
75,2021-02-05 18:58:03,1815,11798055,Accepted,78,Yearn Finance (all vaults),20126.90,10.0,ETH,2020-09-16 23:08:00,10.000000
76,2021-02-06 07:31:02,759,11801426,Accepted,80,Yearn Finance (all vaults),100634.50,50.0,ETH,2020-08-28 06:40:35,50.000000


In [26]:
accepted_claims.amount_eth.describe()

count      26.000000
mean       63.706526
std       192.886730
min         1.174142
25%         6.919051
50%        17.500000
75%        41.450233
max      1000.000000
Name: amount_eth, dtype: float64

## Check ratio of ETH vs USD Liabilities

In [51]:
active_covers = cover_history[cover_history['end_time'] > pd.Timestamp.now()]

In [58]:
active_covers.groupby('currency').sum()['amount_usd']/(active_covers.groupby('currency').sum()['amount_usd'].sum())

currency
DAI    0.801497
ETH    0.198503
Name: amount_usd, dtype: float64

In [52]:
active_covers_no_enzyme_anchor = active_covers[~active_covers['project'].isin(['Anchor', 'Enzyme v3'])]

In [53]:
currency_df = active_covers_no_enzyme_anchor.groupby('currency').sum()

In [54]:
percentages = currency_df['amount_usd']/(currency_df['amount_usd'].sum())
percentages

currency
DAI    0.862858
ETH    0.137142
Name: amount_usd, dtype: float64

In [55]:
currency_df_with_enz_anch = active_covers[active_covers['project'].isin(['Anchor', 'Enzyme v3'])].groupby('currency').sum()

In [56]:
percentages = currency_df_with_enz_anch['amount_usd']/currency_df_with_enz_anch['amount_usd'].sum()
percentages

currency
DAI    0.591042
ETH    0.408958
Name: amount_usd, dtype: float64

## Playground for distributions

In [15]:
randomizer = lognorm.rvs(s=2, loc=0, scale=1, size=10_000)
pd.DataFrame(randomizer).describe()

,0
count,10000.000000
mean,7.140724
std,39.163480
min,0.000854
25%,0.257125
50%,0.977785
75%,3.750230
max,1988.184589


In [16]:
randomizer_2 = lognorm.rvs(s=2, loc=100, scale=1, size=10_000)
pd.DataFrame(randomizer_2).describe()

,0
count,10000.000000
mean,107.328086
std,39.894548
min,100.000863
25%,100.257993
50%,101.010296
75%,103.852078
max,1897.985626
